In [1]:
from datasets import load_dataset

ds = load_dataset("allenai/multinews_sparse_oracle")

/home/hungpv/miniconda3/envs/lightrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this 

In [2]:
ds


DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [3]:
ds["test"].to_json("/home/hungpv/projects/next_work/multinews_test.jsonl")

Creating json from Arrow format: 100%|██████████| 6/6 [00:01<00:00,  5.95ba/s]


91128562

In [5]:
data = []
import json
with open("/home/hungpv/projects/next_work/multinews_test.jsonl",'r') as f:
    for line in f:
    # Mỗi `line` là một chuỗi, ví dụ: '{"document": "...", "summary": "..."}\n'
    # Dùng json.loads() (load string) để chuyển chuỗi thành dictionary
    # Dùng .strip() để xóa các khoảng trắng hoặc ký tự xuống dòng thừa ở đầu/cuối
        parsed_line = json.loads(line.strip())
        data.append(parsed_line)

In [12]:
raw_data = []
for x in data:
    len_doc = len(x['document'].split("|||||"))
    if len_doc >= 5:
        raw_data.append(x['document'])

In [21]:
len(raw_data)
with open("/home/hungpv/projects/next_work/raw_data.json",'w', encoding='utf-8') as f:
    json.dump(raw_data, f, ensure_ascii=False, indent=4)

In [20]:
raw_data[4].split("|||||")


['President Trump and first lady Melania Trump arrive in Tel Aviv. They are flanked by Israel’s president and prime minister and their spouses. (Ilia Yefimovich/Getty Images) \n \n If public appearances by a president and first lady are political theater, first lady Melania Trump and her husband, President Trump, miiight want to work on their choreography. \n \n A brief moment caught on camera during the first couple’s arrival at the Tel Aviv airport appears to show the president reaching behind him for his wife’s hand — and she seemingly swatting it away. Or maybe, upon close inspection of the footage, perhaps she just missed grabbing his outstretched hand, like a trapeze artist flubbing the catch. \n \n Rebuffed, POTUS smooths his tie. \n \n The couple’s efforts at hand-holding have long fascinated White House watchers. (“Donald Trump ‘avoids holding Melania’s hand in public because he wants to look like a presidential alpha,’ says body language expert,” was the headline of a Daily M

In [22]:
!pip install bertopic

In [54]:
# -*- coding: utf-8 -*-
"""
File: phan_cum_van_ban_bge_m3.py
Mô tả: Script hoàn chỉnh để phân cụm văn bản tiếng Việt bằng BERTopic,
      sử dụng mô hình embedding BGE-M3 để nâng cao chất lượng.
Cách chạy:
1. Mở terminal, chạy: pip install bertopic[visualization] sentence-transformers --quiet
2. Chạy lệnh: python phan_cum_van_ban_bge_m3.py
"""
import json

# --- Import các thư viện cần thiết ---
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer # <<< THÊM IMPORT NÀY
import os

# def thuc_hien_phan_cum_voi_bge():
    # """
    # Hàm chính thực hiện toàn bộ quy trình phân cụm văn bản với BGE-M3.
    # """
print("--- BẮT ĐẦU QUÁ TRÌNH PHÂN CỤM VỚI BERTOPIC VÀ BGE-M3 ---")

# --- Chuẩn bị dữ liệu ---
print("\n[INFO] Đang chuẩn bị dữ liệu mẫu...")

with open("/home/hungpv/projects/next_work/raw_data.json",'r') as f:
    all_data = json.load(f)
all_data = all_data[:10]
docs = []
for item in all_data:
    docs.extend(item.split("|||||"))
print(f"[INFO] Đã tải {len(docs)} văn bản mẫu.")

# --- Bước thay đổi quan trọng: Khởi tạo mô hình Embedding BGE-M3 ---
print("\n[INFO] Đang tải mô hình embedding BAAI/bge-m3 từ Hugging Face...")
print("[HINT] Quá trình này có thể mất vài phút cho lần chạy đầu tiên vì model khá lớn (~2.2GB).")

# device="cuda" nếu bạn có GPU, "cpu" nếu không có
embedding_model = SentenceTransformer("BAAI/bge-m3", device="cuda")
print("[SUCCESS] Tải xong mô hình embedding.")


# --- Khởi tạo và Huấn luyện mô hình BERTopic với embedding model tùy chỉnh ---
print("\n[INFO] Đang khởi tạo BERTopic và huấn luyện với mô hình BGE-M3...")

# Thay vì `language="multilingual"`, chúng ta truyền trực tiếp đối tượng `embedding_model`
topic_model = BERTopic(
    embedding_model=embedding_model, # <<< THAY ĐỔI QUAN TRỌNG
    verbose=True,
    min_topic_size=2
)

# Huấn luyện mô hình
topics, probs = topic_model.fit_transform(docs)
print("\n[SUCCESS] Hoàn thành huấn luyện mô hình.")

# --- Các bước còn lại (Khám phá, Trực quan hóa, Lưu trữ) giữ nguyên như cũ ---

# Khám phá kết quả
print("\n--- KẾT QUẢ PHÂN CỤM ---")
topic_info = topic_model.get_topic_info()
print("\n[INFO] Thông tin tổng quan các chủ đề tìm thấy:")
print(topic_info)

# Trực quan hóa và lưu kết quả
print("\n[INFO] Đang tạo các biểu đồ trực quan và lưu thành file HTML...")
output_dir = "ket_qua_bertopic_bge_m3"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
try:
    fig_topics = topic_model.visualize_topics()
    fig_topics.write_html(os.path.join(output_dir, "cac_cum_chu_de.html"))
    print(f"[SUCCESS] Đã lưu biểu đồ cụm chủ đề tại: {os.path.join(output_dir, 'cac_cum_chu_de.html')}")

except Exception as e:
    print(f"[ERROR] Không thể tạo biểu đồ. Lỗi: {e}")

# Lưu mô hình
print("\n[INFO] Đang lưu mô hình đã huấn luyện...")
model_save_path = "mo_hinh_bertopic_bge_m3"
topic_model.save(model_save_path, serialization="safetensors")
print(f"[SUCCESS] Đã lưu mô hình tại thư mục: {model_save_path}")

print("\n--- HOÀN TẤT TOÀN BỘ QUÁ TRÌNH ---")


# if __name__ == "__main__":
# thuc_hien_phan_cum_voi_bge()

--- BẮT ĐẦU QUÁ TRÌNH PHÂN CỤM VỚI BERTOPIC VÀ BGE-M3 ---

[INFO] Đang chuẩn bị dữ liệu mẫu...
[INFO] Đã tải 55 văn bản mẫu.

[INFO] Đang tải mô hình embedding BAAI/bge-m3 từ Hugging Face...
[HINT] Quá trình này có thể mất vài phút cho lần chạy đầu tiên vì model khá lớn (~2.2GB).


2025-06-23 08:08:15,543 - BERTopic - Embedding - Transforming documents to embeddings.


[SUCCESS] Tải xong mô hình embedding.

[INFO] Đang khởi tạo BERTopic và huấn luyện với mô hình BGE-M3...


Batches: 100%|██████████| 2/2 [00:24<00:00, 12.17s/it]
2025-06-23 08:08:39,903 - BERTopic - Embedding - Completed ✓
2025-06-23 08:08:39,903 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-23 08:08:40,002 - BERTopic - Dimensionality - Completed ✓
2025-06-23 08:08:40,003 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-23 08:08:40,008 - BERTopic - Cluster - Completed ✓
2025-06-23 08:08:40,010 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-23 08:08:40,084 - BERTopic - Representation - Completed ✓



[SUCCESS] Hoàn thành huấn luyện mô hình.

--- KẾT QUẢ PHÂN CỤM ---

[INFO] Thông tin tổng quan các chủ đề tìm thấy:
   Topic  Count                          Name  \
0      0      6               0_the_of_and_to   
1      1      6            1_amc_to_the_movie   
2      2      6          2_the_clinton_and_to   
3      3      6       3_color_pantone_the_and   
4      4      6        4_trump_the_melania_to   
5      5      5  5_brewing_company_co_brewery   
6      6      5       6_alto_palo_housing_and   
7      7      5        7_disney_the_wars_star   
8      8      5             8_the_death_in_of   
9      9      5          9_swift_the_to_perry   

                                      Representation  \
0  [the, of, and, to, storm, in, hurricane, said,...   
1  [amc, to, the, movie, that, in, and, theaters,...   
2  [the, clinton, and, to, of, in, obama, he, her...   
3  [color, pantone, the, and, green, of, in, to, ...   
4  [trump, the, melania, to, and, you, of, that, ...   
5  [bre

In [47]:
len(topic_info['Representative_Docs'][1])

3

In [45]:
raw_data[0].split("|||||")


['CLOSE Sierra Nevada Brewing Co. is releasing a special edition Resilience Butte County IPA that will send 100% of the proceeds to Camp Fire relief efforts. USA TODAY \n \n Sierra Nevada Brewing Co. in Chico, California, in making Resilience Butte County Proud IPA, the proceeds of which will support Camp Fire relief efforts. (Photo: Sierra Nevada Brewing Co.) \n \n The deadliest fire in California history has set into motion the largest charitable collaboration ever among the nation\'s breweries, as more than 1,200 have signed on to brew a beer to raise funds for those affected by the Camp Fire in Northern California. \n \n Sierra Nevada Brewing Co., which is located in Chico, California, part of Butte County but west of the areas hardest hit by the fire, fed first responders and displaced residents during the fire and handed out clothing, too. \n \n But Ken Grossman, who founded the brewery in 1980, wanted to do more. He decided to brew a special beer, Resilience Butte County Proud I

In [57]:
# Giả sử bạn đã có `topic_model` đã được huấn luyện và danh sách `docs`
# ...
# topic_model.fit_transform(docs)

# Lấy thông tin chi tiết của từng văn bản
document_info = topic_model.get_document_info(docs)

# In ra vài dòng đầu của DataFrame để xem cấu trúc
print("--- BẢNG THÔNG TIN CHI TIẾT CỦA TỪNG VĂN BẢN ---")
print(document_info.head())

--- BẢNG THÔNG TIN CHI TIẾT CỦA TỪNG VĂN BẢN ---
                                            Document  Topic  \
0  CLOSE Sierra Nevada Brewing Co. is releasing a...      5   
1   Sierra Nevada Brewery in Chico is brewing up ...      5   
2   November 21, 2018 \n \n The Camp Fire–the mos...      5   
3   In early November, the Camp Fire swept across...      5   
4   (Packaged in cans by Sierra Nevada, and draug...      5   

                           Name  \
0  5_brewing_company_co_brewery   
1  5_brewing_company_co_brewery   
2  5_brewing_company_co_brewery   
3  5_brewing_company_co_brewery   
4  5_brewing_company_co_brewery   

                                      Representation  \
0  [brewing, company, co, brewery, beer, fire, re...   
1  [brewing, company, co, brewery, beer, fire, re...   
2  [brewing, company, co, brewery, beer, fire, re...   
3  [brewing, company, co, brewery, beer, fire, re...   
4  [brewing, company, co, brewery, beer, fire, re...   

                        

In [58]:
document_info

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,CLOSE Sierra Nevada Brewing Co. is releasing a...,5,5_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, fire, re...","[ November 21, 2018 \n \n The Camp Fire–the mo...",brewing - company - co - brewery - beer - fire...,0.973211,True
1,Sierra Nevada Brewery in Chico is brewing up ...,5,5_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, fire, re...","[ November 21, 2018 \n \n The Camp Fire–the mo...",brewing - company - co - brewery - beer - fire...,1.000000,False
2,"November 21, 2018 \n \n The Camp Fire–the mos...",5,5_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, fire, re...","[ November 21, 2018 \n \n The Camp Fire–the mo...",brewing - company - co - brewery - beer - fire...,0.946601,True
3,"In early November, the Camp Fire swept across...",5,5_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, fire, re...","[ November 21, 2018 \n \n The Camp Fire–the mo...",brewing - company - co - brewery - beer - fire...,1.000000,False
4,"(Packaged in cans by Sierra Nevada, and draug...",5,5_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, fire, re...","[ November 21, 2018 \n \n The Camp Fire–the mo...",brewing - company - co - brewery - beer - fire...,0.995480,True
5,PALO ALTO -- You're a well-paid professional. ...,6,6_alto_palo_housing_and,"[alto, palo, housing, and, to, the, in, of, co...",[ In what’s becoming a regular story in the Ba...,alto - palo - housing - and - to - the - in - ...,1.000000,False
6,"For the past two years, Kate Downing has been...",6,6_alto_palo_housing_and,"[alto, palo, housing, and, to, the, in, of, co...",[ In what’s becoming a regular story in the Ba...,alto - palo - housing - and - to - the - in - ...,0.816031,True
7,In what’s becoming a regular story in the Bay...,6,6_alto_palo_housing_and,"[alto, palo, housing, and, to, the, in, of, co...",[ In what’s becoming a regular story in the Ba...,alto - palo - housing - and - to - the - in - ...,1.000000,True
8,Letter of Resignation from the Palo Alto Plan...,6,6_alto_palo_housing_and,"[alto, palo, housing, and, to, the, in, of, co...",[ In what’s becoming a regular story in the Ba...,alto - palo - housing - and - to - the - in - ...,1.000000,True
9,PALO ALTO -- Palantir Technologies is quietly...,6,6_alto_palo_housing_and,"[alto, palo, housing, and, to, the, in, of, co...",[ In what’s becoming a regular story in the Ba...,alto - palo - housing - and - to - the - in - ...,0.717830,False


In [61]:
raw_data[2].split("|||||")


['For years now, movie theater operators have been trying to dissuade their customers from chatting on cell phones or texting during shows. Some issue stern pre-show warnings that anyone caught texting or talking will be shown the door. \n \n It\'s not surprising then that the new chief executive and president of AMC Theaters, Adam M. Aron, caused a stir by suggesting that nation\'s second-largest movie theater company would consider the idea of making certain auditoriums "more texting friendly" in a bid to get millennials to go to the movies more often. \n \n "When you tell a 22-year-old to turn off the phone, don\'t ruin the movie, they hear please cut off your left arm above the elbow," said Aron in an interview with Variety published online Wednesday. \n \n "You can\'t tell a 22-year-old to turn off their cellphone. That\'s not how they live," said Aron, who hailed from outside the movie business before taking the reins of parent company AMC Entertainment Holdings (AMC) in January.

In [2]:
# -*- coding: utf-8 -*-
"""
File: run_full_pipeline.py
Mô tả: Một quy trình hoàn chỉnh và mạnh mẽ để phân cụm văn bản tiếng Việt.
       Script này tích hợp các phương pháp tốt nhất:
       1. Tự động chia nhỏ (chunking) các văn bản dài.
       2. Sử dụng GPU và mô hình embedding BGE-M3 hiệu năng cao.
       3. Tạo embedding theo batch để tối ưu bộ nhớ và tốc độ.
       4. Lưu lại toàn bộ kết quả một cách có hệ thống.

Cách chạy:
1. Mở terminal, cài đặt các thư viện cần thiết:
   pip install bertopic[visualization] sentence-transformers transformers --quiet
2. Chỉnh sửa các tham số trong "BƯỚC 1" bên dưới cho phù hợp.
3. Chạy script:
   python run_full_pipeline.py
"""

# --- Import các thư viện ---
import json
import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch # Dùng để kiểm tra GPU

# --- HÀM TIỆN ÍCH ---
def split_into_chunks(text, tokenizer, chunk_size=512, overlap=50):
    """Hàm chia một văn bản dài thành các đoạn nhỏ hơn dựa trên số token."""
    tokens = tokenizer.encode(text)
    
    chunks_text = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        if not chunk_tokens:
            continue
        # Decode ngược lại thành text, bỏ qua các token đặc biệt (ví dụ [CLS], [SEP])
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks_text.append(chunk_text)
    return chunks_text

# def main():
"""Hàm chính thực thi toàn bộ quy trình."""

# ==============================================================================
# BƯỚC 1: CÁC THAM SỐ CẤU HÌNH (Chỉnh sửa tại đây)
# ==============================================================================
# --- Cấu hình file và dữ liệu ---
DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
MAX_DOCS_TO_PROCESS = None  # Đặt là None để xử lý toàn bộ file, hoặc đặt số lượng (ví dụ 8000)

# --- Cấu hình chia nhỏ văn bản (Chunking) ---
CHUNK_SIZE = 512  # Số token tối đa trong một chunk
CHUNK_OVERLAP = 50 # Số token gối lên nhau giữa các chunk
LONG_DOC_THRESHOLD = 450 # Số token, văn bản nào dài hơn sẽ bị chia nhỏ

# --- Cấu hình mô hình ---
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
BATCH_SIZE = 64  # Giảm nếu bị lỗi "out of memory", tăng để nhanh hơn nếu VRAM lớn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Cấu hình BERTopic ---
MIN_TOPIC_SIZE = 5 # Số văn bản tối thiểu để tạo thành một chủ đề

# --- Cấu hình lưu trữ ---
OUTPUT_DIR = "results_pipeline"
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, "bertopic_model_final")

print("--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM HOÀN CHỈNH ---")
print(f"Thiết bị tính toán được sử dụng: {DEVICE.upper()}")

# Tạo thư mục output nếu chưa có
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================================================================
# BƯỚC 2: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU (CÓ CHUNKING)
# ==============================================================================
print(f"\n[BƯỚC 2] Đang tải và xử lý dữ liệu từ: {DATA_FILE_PATH}...")
try:
    with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
        all_data = json.load(f)
except Exception as e:
    print(f"[LỖI] Không thể đọc file dữ liệu. Lỗi: {e}")
    # return

if MAX_DOCS_TO_PROCESS is not None:
    all_data = all_data[:MAX_DOCS_TO_PROCESS]

docs = []
# Tải tokenizer một lần để sử dụng cho việc chia nhỏ
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)

for item in all_data:
    if isinstance(item, str):
        sub_docs = item.split("|||||")
        for doc in sub_docs:
            doc = doc.strip()
            if not doc: continue
            
            # Kiểm tra độ dài bằng token
            num_tokens = len(tokenizer.encode(doc))
            
            if num_tokens > LONG_DOC_THRESHOLD:
                chunks = split_into_chunks(doc, tokenizer, CHUNK_SIZE, CHUNK_OVERLAP)
                docs.extend(chunks)
            else:
                docs.append(doc)

if not docs:
    print("[LỖI] Không tìm thấy văn bản nào hợp lệ sau khi xử lý. Vui lòng kiểm tra lại file dữ liệu.")
    # return
    
print(f"[INFO] Tổng số văn bản/đoạn văn sau khi xử lý: {len(docs)}")

# ==============================================================================
# BƯỚC 3: TẢI MÔ HÌNH EMBEDDING
# ==============================================================================
print(f"\n[BƯỚC 3] Đang tải mô hình embedding: {EMBEDDING_MODEL_NAME}...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)
print("[SUCCESS] Tải xong mô hình embedding.")

# ==============================================================================
# BƯỚC 4: TẠO EMBEDDINGS (THEO BATCH)
# ==============================================================================
print(f"\n[BƯỚC 4] Đang tạo embeddings với batch size = {BATCH_SIZE}...")
embeddings = embedding_model.encode(
    docs,
    batch_size=BATCH_SIZE,
    show_progress_bar=True
)
print(f"[SUCCESS] Đã tạo xong embeddings với shape: {embeddings.shape}")

# ==============================================================================
# BƯỚC 5: HUẤN LUYỆN MÔ HÌNH BERTOPIC
# ==============================================================================
print(f"\n[BƯỚC 5] Đang huấn luyện BERTopic với min_topic_size = {MIN_TOPIC_SIZE}...")
topic_model = BERTopic(verbose=True, min_topic_size=MIN_TOPIC_SIZE)
topics, probs = topic_model.fit_transform(docs, embeddings)
print("[SUCCESS] Hoàn thành huấn luyện mô hình.")

# ==============================================================================
# BƯỚC 6: PHÂN TÍCH VÀ LƯU KẾT QUẢ
# ==============================================================================
print("\n[BƯỚC 6] Đang phân tích và lưu kết quả...")
# Lưu thông tin tổng quan các chủ đề
topic_info_df = topic_model.get_topic_info()
print("\n[INFO] Thông tin tổng quan các chủ đề tìm thấy:")
print(topic_info_df.head(15))
topic_info_df.to_csv(os.path.join(OUTPUT_DIR, "tong_quan_chu_de.csv"), index=False, encoding='utf-8-sig')

# Lưu thông tin chi tiết của từng văn bản
doc_info_df = topic_model.get_document_info(docs)
doc_info_df.to_csv(os.path.join(OUTPUT_DIR, "chi_tiet_van_ban_theo_chu_de.csv"), index=False, encoding='utf-8-sig')
print(f"[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: {OUTPUT_DIR}")

# ==============================================================================
# BƯỚC 7: TRỰC QUAN HÓA
# ==============================================================================
print("\n[BƯỚC 7] Đang tạo các biểu đồ trực quan...")
try:
    # Biểu đồ các cụm chủ đề
    fig_topics = topic_model.visualize_topics()
    fig_topics.write_html(os.path.join(OUTPUT_DIR, "interactive_topics_map.html"))

    # Biểu đồ các từ khóa của từng chủ đề
    fig_barchart = topic_model.visualize_barchart(top_n_topics=12)
    fig_barchart.write_html(os.path.join(OUTPUT_DIR, "topic_keywords_barchart.html"))
    print(f"[SUCCESS] Đã lưu các biểu đồ ra file HTML trong thư mục: {OUTPUT_DIR}")
except Exception as e:
    print(f"[CẢNH BÁO] Không thể tạo biểu đồ. Lỗi: {e}")

# ==============================================================================
# BƯỚC 8: LƯU MÔ HÌNH ĐÃ HUẤN LUYỆN
# ==============================================================================
print(f"\n[BƯỚC 8] Đang lưu mô hình đã huấn luyện tại: {MODEL_SAVE_PATH}...")
topic_model.save(MODEL_SAVE_PATH, serialization="safetensors")
print("[SUCCESS] Đã lưu mô hình thành công.")

print("\n--- QUY TRÌNH HOÀN TẤT ---")

# if __name__ == "__main__":
#     main()

--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM HOÀN CHỈNH ---
Thiết bị tính toán được sử dụng: CUDA

[BƯỚC 2] Đang tải và xử lý dữ liệu từ: /home/hungpv/projects/next_work/raw_data.json...


Token indices sequence length is longer than the specified maximum sequence length for this model (10351 > 8192). Running this sequence through the model will result in indexing errors


[INFO] Tổng số văn bản/đoạn văn sau khi xử lý: 8374

[BƯỚC 3] Đang tải mô hình embedding: BAAI/bge-m3...
[SUCCESS] Tải xong mô hình embedding.

[BƯỚC 4] Đang tạo embeddings với batch size = 64...


Batches: 100%|██████████| 131/131 [02:45<00:00,  1.27s/it]
2025-06-23 09:31:32,323 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[SUCCESS] Đã tạo xong embeddings với shape: (8374, 1024)

[BƯỚC 5] Đang huấn luyện BERTopic với min_topic_size = 5...


2025-06-23 09:31:54,546 - BERTopic - Dimensionality - Completed ✓
2025-06-23 09:31:54,547 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-23 09:31:54,950 - BERTopic - Cluster - Completed ✓
2025-06-23 09:31:54,956 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-23 09:31:57,442 - BERTopic - Representation - Completed ✓


[SUCCESS] Hoàn thành huấn luyện mô hình.

[BƯỚC 6] Đang phân tích và lưu kết quả...

[INFO] Thông tin tổng quan các chủ đề tìm thấy:
    Topic  Count                                     Name  \
0      -1    413            -1_trump_russia_clinton_would   
1       0    327             0_brain_mice_results_methods   
2       1    173         1_harvey_texas_houston_hurricane   
3       2     87                2_ebola_epidemic_45_virus   
4       3     77          3_sandy_hurricane_updated_storm   
5       4     74        4_plane_malaysia_flight_malaysian   
6       5     66         5_punishment_footnote_430_eighth   
7       6     62                   6_trip_69_trump_trumps   
8       7     61             7_north_korea_missile_korean   
9       8     61       8_hampshire_sanders_primary_kasich   
10      9     56           9_syrian_strike_syria_chemical   
11     10     55          10_duncan_ebola_dallas_hospital   
12     11     53             11_players_anthem_nfl_eagles   
13     12    

In [1]:
# -*- coding: utf-8 -*-
"""
File: run_full_pipeline_v2.py
Mô tả: Phiên bản hoàn thiện nhất của quy trình phân cụm văn bản.
       - Sửa lỗi logic: Kiểm tra và chia nhỏ văn bản dựa trên SỐ LƯỢNG TOKEN.
       - Tối ưu Chunk Size: Tận dụng khả năng xử lý văn bản dài của model.
"""

# --- Import các thư viện ---
import json
import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch

# --- HÀM TIỆN ÍCH ---
def split_into_chunks(text, tokenizer, chunk_size, overlap):
    """Hàm chia một văn bản dài thành các đoạn nhỏ hơn dựa trên số token."""
    tokens = tokenizer.encode(text, add_special_tokens=False) # Không cần thêm token đặc biệt khi chunk
    
    chunks_text = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        if not chunk_tokens:
            continue
        # Decode ngược lại thành text
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks_text.append(chunk_text)
    return chunks_text

# def main():
#     """Hàm chính thực thi toàn bộ quy trình."""

#     # ==============================================================================
#     # BƯỚC 1: CÁC THAM SỐ CẤU HÌNH (Chỉnh sửa tại đây)
#     # ==============================================================================
print("--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM TỐI ƯU ---")

# --- Cấu hình file và dữ liệu ---
DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
MAX_DOCS_TO_PROCESS = 10  # Đặt là None để xử lý toàn bộ file

# --- Cấu hình chia nhỏ văn bản (Chunking) ---
# Tăng CHUNK_SIZE để giữ lại nhiều ngữ cảnh hơn.
# Con số này nên nhỏ hơn giới hạn của model (8192) để an toàn.
CHUNK_SIZE = 8192
CHUNK_OVERLAP = 200 # Gối lên nhau 200 token
# Ngưỡng token: Bất kỳ văn bản nào dài hơn ngưỡng này sẽ bị chia nhỏ
LONG_DOC_TOKEN_THRESHOLD = CHUNK_SIZE 

# --- Cấu hình mô hình ---
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
BATCH_SIZE = 32  # Giảm batch size khi chunk size tăng để cân bằng VRAM
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Cấu hình BERTopic ---
MIN_TOPIC_SIZE = 5

# --- Cấu hình lưu trữ ---
OUTPUT_DIR = "results_pipeline_v2"
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, "bertopic_model_final")

print(f"Thiết bị tính toán: {DEVICE.upper()} | Kích thước chunk: {CHUNK_SIZE} | Batch size: {BATCH_SIZE}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================================================================
# BƯỚC 2: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU (LOGIC CHUNKING ĐÃ SỬA)
# ==============================================================================
print(f"\n[BƯỚC 2] Đang tải và xử lý dữ liệu từ: {DATA_FILE_PATH}...")
try:
    with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
        all_data = json.load(f)
except Exception as e:
    print(f"[LỖI] Không thể đọc file dữ liệu. Lỗi: {e}")
    # return

if MAX_DOCS_TO_PROCESS is not None:
    all_data = all_data[:MAX_DOCS_TO_PROCESS]

docs = []
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)

for item in all_data:
    if isinstance(item, str):
        sub_docs = item.split("|||||")
        for doc in sub_docs:
            doc = doc.strip()
            # if len(doc) < 50: continue # Bỏ qua các văn bản quá ngắn

            # === SỬA LỖI LOGIC QUAN TRỌNG: KIỂM TRA BẰNG SỐ TOKEN ===
            num_tokens = len(tokenizer.encode(doc, add_special_tokens=False))
            
            if num_tokens > LONG_DOC_TOKEN_THRESHOLD:
                chunks = split_into_chunks(doc, tokenizer, CHUNK_SIZE, CHUNK_OVERLAP)
                docs.extend(chunks)
            else:
                docs.append(doc)

if not docs:
    print("[LỖI] Không tìm thấy văn bản nào hợp lệ sau khi xử lý.")
    # return
    
print(f"[INFO] Tổng số văn bản/đoạn văn sau khi xử lý: {len(docs)}")

# ==============================================================================
# CÁC BƯỚC 3, 4, 5, 6, 7, 8: Giữ nguyên như cũ
# (Tải model, Tạo Embeddings, Huấn luyện, Lưu kết quả, Trực quan hóa, Lưu model)
# ==============================================================================
print(f"\n[BƯỚC 3] Đang tải mô hình embedding: {EMBEDDING_MODEL_NAME}...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)

print(f"\n[BƯỚC 4] Đang tạo embeddings với batch size = {BATCH_SIZE}...")
embeddings = embedding_model.encode(docs, batch_size=BATCH_SIZE, show_progress_bar=True)

print(f"\n[BƯỚC 5] Đang huấn luyện BERTopic với min_topic_size = {MIN_TOPIC_SIZE}...")
topic_model = BERTopic(verbose=True, min_topic_size=MIN_TOPIC_SIZE)
topics, probs = topic_model.fit_transform(docs, embeddings)


# ==============================================================================
# BƯỚC 6: PHÂN TÍCH VÀ LƯU KẾT QUẢ
# ==============================================================================
print("\n[BƯỚC 6] Đang phân tích và lưu kết quả...")
# Lưu thông tin tổng quan các chủ đề
topic_info_df = topic_model.get_topic_info()
print("\n[INFO] Thông tin tổng quan các chủ đề tìm thấy:")
print(topic_info_df.head(15))
topic_info_df.to_csv(os.path.join(OUTPUT_DIR, "tong_quan_chu_de.csv"), index=False, encoding='utf-8-sig')

# Lưu thông tin chi tiết của từng văn bản
doc_info_df = topic_model.get_document_info(docs)
doc_info_df.to_csv(os.path.join(OUTPUT_DIR, "chi_tiet_van_ban_theo_chu_de.csv"), index=False, encoding='utf-8-sig')
print(f"[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: {OUTPUT_DIR}")

# ==============================================================================
# BƯỚC 7: TRỰC QUAN HÓA
# ==============================================================================
print("\n[BƯỚC 7] Đang tạo các biểu đồ trực quan...")
try:
    # Biểu đồ các cụm chủ đề
    fig_topics = topic_model.visualize_topics()
    fig_topics.write_html(os.path.join(OUTPUT_DIR, "interactive_topics_map.html"))

    # Biểu đồ các từ khóa của từng chủ đề
    fig_barchart = topic_model.visualize_barchart(top_n_topics=12)
    fig_barchart.write_html(os.path.join(OUTPUT_DIR, "topic_keywords_barchart.html"))
    print(f"[SUCCESS] Đã lưu các biểu đồ ra file HTML trong thư mục: {OUTPUT_DIR}")
except Exception as e:
    print(f"[CẢNH BÁO] Không thể tạo biểu đồ. Lỗi: {e}")

# ==============================================================================
# BƯỚC 8: LƯU MÔ HÌNH ĐÃ HUẤN LUYỆN
# ==============================================================================
print(f"\n[BƯỚC 8] Đang lưu mô hình đã huấn luyện tại: {MODEL_SAVE_PATH}...")
topic_model.save(MODEL_SAVE_PATH, serialization="safetensors")
print("[SUCCESS] Đã lưu mô hình thành công.")

print("\n--- QUY TRÌNH HOÀN TẤT ---")

print("\n--- QUY TRÌNH HOÀN TẤT ---")


/home/hungpv/miniconda3/envs/lightrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM TỐI ƯU ---
Thiết bị tính toán: CUDA | Kích thước chunk: 8192 | Batch size: 32

[BƯỚC 2] Đang tải và xử lý dữ liệu từ: /home/hungpv/projects/next_work/raw_data.json...


Token indices sequence length is longer than the specified maximum sequence length for this model (10349 > 8192). Running this sequence through the model will result in indexing errors


[INFO] Tổng số văn bản/đoạn văn sau khi xử lý: 57

[BƯỚC 3] Đang tải mô hình embedding: BAAI/bge-m3...

[BƯỚC 4] Đang tạo embeddings với batch size = 32...


Batches: 100%|██████████| 2/2 [00:24<00:00, 12.24s/it]
2025-06-23 09:39:40,453 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm



[BƯỚC 5] Đang huấn luyện BERTopic với min_topic_size = 5...


2025-06-23 09:39:49,365 - BERTopic - Dimensionality - Completed ✓
2025-06-23 09:39:49,365 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-23 09:39:49,371 - BERTopic - Cluster - Completed ✓
2025-06-23 09:39:49,374 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-23 09:39:49,448 - BERTopic - Representation - Completed ✓



[BƯỚC 6] Đang phân tích và lưu kết quả...

[INFO] Thông tin tổng quan các chủ đề tìm thấy:
   Topic  Count                          Name  \
0     -1     11               -1_the_in_to_of   
1      0     23               0_the_to_and_of   
2      1     11               1_the_to_and_in   
3      2      6  2_brewing_company_co_brewery   
4      3      6       3_color_the_and_pantone   

                                      Representation  \
0  [the, in, to, of, and, palo, death, for, alto,...   
1   [the, to, and, of, in, trump, that, for, is, on]   
2  [the, to, and, in, of, that, disney, amc, movi...   
3  [brewing, company, co, brewery, beer, the, res...   
4  [color, the, and, pantone, of, in, to, green, ...   

                                 Representative_Docs  
0  [Global figures \n \n At least 1,032 people we...  
1  [(CNN) -- The second night of the Democratic N...  
2  [AMC Theatres is threatening legal action agai...  
3  [CLOSE Sierra Nevada Brewing Co. is releasing ...  
4

In [2]:
doc_info_df

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,CLOSE Sierra Nevada Brewing Co. is releasing a...,2,2_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, the, res...",[CLOSE Sierra Nevada Brewing Co. is releasing ...,brewing - company - co - brewery - beer - the ...,1.000000,True
1,Sierra Nevada Brewery in Chico is brewing up a...,2,2_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, the, res...",[CLOSE Sierra Nevada Brewing Co. is releasing ...,brewing - company - co - brewery - beer - the ...,1.000000,False
2,"November 21, 2018 \n \n The Camp Fire–the most...",2,2_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, the, res...",[CLOSE Sierra Nevada Brewing Co. is releasing ...,brewing - company - co - brewery - beer - the ...,1.000000,False
3,"In early November, the Camp Fire swept across ...",2,2_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, the, res...",[CLOSE Sierra Nevada Brewing Co. is releasing ...,brewing - company - co - brewery - beer - the ...,1.000000,False
4,"(Packaged in cans by Sierra Nevada, and draugh...",2,2_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, the, res...",[CLOSE Sierra Nevada Brewing Co. is releasing ...,brewing - company - co - brewery - beer - the ...,1.000000,True
5,st Brewing Company Southern Star Brewing Compa...,2,2_brewing_company_co_brewery,"[brewing, company, co, brewery, beer, the, res...",[CLOSE Sierra Nevada Brewing Co. is releasing ...,brewing - company - co - brewery - beer - the ...,1.000000,True
6,PALO ALTO -- You're a well-paid professional. ...,-1,-1_the_in_to_of,"[the, in, to, of, and, palo, death, for, alto,...","[Global figures \n \n At least 1,032 people we...",the - in - to - of - and - palo - death - for ...,0.000000,True
7,"For the past two years, Kate Downing has been ...",-1,-1_the_in_to_of,"[the, in, to, of, and, palo, death, for, alto,...","[Global figures \n \n At least 1,032 people we...",the - in - to - of - and - palo - death - for ...,0.000000,True
8,In what’s becoming a regular story in the Bay ...,-1,-1_the_in_to_of,"[the, in, to, of, and, palo, death, for, alto,...","[Global figures \n \n At least 1,032 people we...",the - in - to - of - and - palo - death - for ...,0.000000,False
9,Letter of Resignation from the Palo Alto Plann...,0,0_the_to_and_of,"[the, to, and, of, in, trump, that, for, is, on]",[(CNN) -- The second night of the Democratic N...,the - to - and - of - in - trump - that - for ...,0.672800,False


In [ ]:
# -*- coding: utf-8 -*-
"""
File: run_truncation_pipeline.py
Mô tả: Quy trình phân cụm văn bản bằng phương pháp Truncation (Cắt ngắn).
       Mỗi văn bản dài sẽ được cắt để lấy N token đầu tiên làm đại diện.
"""

import json
import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch

# def main():
    # ==============================================================================
    # BƯỚC 1: CÁC THAM SỐ CẤU HÌNH
# ==============================================================================
print("--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM BẰNG PHƯƠNG PHÁP TRUNCATION ---")

DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
EMBEDDING_MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"

# Giới hạn token tối đa cho mỗi văn bản. 8192 là giới hạn của BGE-M3
MAX_TOKENS_PER_DOC = 32000

BATCH_SIZE = 16
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MIN_TOPIC_SIZE = 5
OUTPUT_DIR = "results_truncation_pipeline"

print(f"Thiết bị: {DEVICE.upper()} | Giới hạn token mỗi văn bản: {MAX_TOKENS_PER_DOC}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================================================================
# BƯỚC 2: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU (THEO LOGIC TRUNCATION)
# ==============================================================================
print(f"\n[BƯỚC 2] Đang tải và xử lý dữ liệu từ: {DATA_FILE_PATH}...")
try:
    with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
        all_data = json.load(f)
except Exception as e:
    print(f"[LỖI] Không thể đọc file dữ liệu. Lỗi: {e}")
    # return

docs = []
original_docs_map = [] # Dùng để map lại sau này nếu cần
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)

for index, item in enumerate(all_data):
    if isinstance(item, str):
        sub_docs = item.split("|||||")
        for sub_doc in sub_docs:
            sub_doc = sub_doc.strip()
            # if len(sub_doc) < 50: continue

            # Tokenize văn bản
            tokens = tokenizer.encode(sub_doc, add_special_tokens=False)
            
            # Cắt ngắn nếu dài hơn giới hạn
            if len(tokens) > MAX_TOKENS_PER_DOC:
                tokens = tokens[:MAX_TOKENS_PER_DOC]
            
            # Decode ngược lại thành văn bản
            processed_doc = tokenizer.decode(tokens)
            docs.append(processed_doc)
            
if not docs:
    print("[LỖI] Không tìm thấy văn bản nào hợp lệ sau khi xử lý.")
    # return
    
print(f"[INFO] Tổng số văn bản sau khi xử lý: {len(docs)}")

# ==============================================================================
# CÁC BƯỚC CÒN LẠI: Tải model, Tạo Embedding, Huấn luyện, Lưu...
# Các bước này hoàn toàn giống với quy trình trước.
# ==============================================================================
print(f"\n[BƯỚC 3] Đang tải mô hình embedding: {EMBEDDING_MODEL_NAME}...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)

print(f"\n[BƯỚC 4] Đang tạo embeddings với batch size = {BATCH_SIZE}...")
embeddings = embedding_model.encode(docs, batch_size=BATCH_SIZE, show_progress_bar=True)

print(f"\n[BƯỚC 5] Đang huấn luyện BERTopic...")
topic_model = BERTopic(verbose=True, min_topic_size=MIN_TOPIC_SIZE)
topics, probs = topic_model.fit_transform(docs, embeddings)

print("\n[BƯỚC 6] Đang lưu kết quả...")
doc_info_df = topic_model.get_document_info(docs)
doc_info_df.to_csv(os.path.join(OUTPUT_DIR, "chi_tiet_van_ban_theo_chu_de.csv"), index=False, encoding='utf-8-sig')
print(f"[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: {OUTPUT_DIR}")

print("\n--- QUY TRÌNH HOÀN TẤT ---")

# if __name__ == "__main__":
#     main()

/home/hungpv/miniconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM BẰNG PHƯƠNG PHÁP TRUNCATION ---
Thiết bị: CUDA | Giới hạn token mỗi văn bản: 32000

[BƯỚC 2] Đang tải và xử lý dữ liệu từ: /home/hungpv/projects/next_work/raw_data.json...


Token indices sequence length is longer than the specified maximum sequence length for this model (131122 > 131072). Running this sequence through the model will result in indexing errors


[INFO] Tổng số văn bản sau khi xử lý: 2153

[BƯỚC 3] Đang tải mô hình embedding: Qwen/Qwen3-Embedding-0.6B...

[BƯỚC 4] Đang tạo embeddings với batch size = 16...


Batches:   0%|          | 0/135 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 61.04 GiB. GPU 0 has a total capacity of 39.39 GiB of which 29.92 GiB is free. Including non-PyTorch memory, this process has 9.45 GiB memory in use. Of the allocated memory 8.95 GiB is allocated by PyTorch, and 20.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
doc_info_df

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,CLOSE Sierra Nevada Brewing Co. is releasing a...,-1,-1_brewing_to_the_in,"[brewing, to, the, in, and, of, that, was, is,...","[Update number 14 - July 8, 2015 Today, July 8...",brewing - to - the - in - and - of - that - wa...,0.000000,False
1,Sierra Nevada Brewery in Chico is brewing up a...,-1,-1_brewing_to_the_in,"[brewing, to, the, in, and, of, that, was, is,...","[Update number 14 - July 8, 2015 Today, July 8...",brewing - to - the - in - and - of - that - wa...,0.000000,False
2,"November 21, 2018 The Camp Fire–the most devas...",-1,-1_brewing_to_the_in,"[brewing, to, the, in, and, of, that, was, is,...","[Update number 14 - July 8, 2015 Today, July 8...",brewing - to - the - in - and - of - that - wa...,0.000000,False
3,"In early November, the Camp Fire swept across ...",-1,-1_brewing_to_the_in,"[brewing, to, the, in, and, of, that, was, is,...","[Update number 14 - July 8, 2015 Today, July 8...",brewing - to - the - in - and - of - that - wa...,0.000000,False
4,"(Packaged in cans by Sierra Nevada, and draugh...",-1,-1_brewing_to_the_in,"[brewing, to, the, in, and, of, that, was, is,...","[Update number 14 - July 8, 2015 Today, July 8...",brewing - to - the - in - and - of - that - wa...,0.000000,False
...,...,...,...,...,...,...,...,...
2148,"Rep. Steve Cohen, a Tennessee Democrat, attrac...",78,78_weiner_weiners_abedin_anthony,"[weiner, weiners, abedin, anthony, sexting, hi...",[Anthony Weiner will finally be cut off. The p...,weiner - weiners - abedin - anthony - sexting ...,0.483773,False
2149,Porn Star Says Weiner Sent Her a Private Messa...,78,78_weiner_weiners_abedin_anthony,"[weiner, weiners, abedin, anthony, sexting, hi...",[Anthony Weiner will finally be cut off. The p...,weiner - weiners - abedin - anthony - sexting ...,0.288696,False
2150,Rep. Anthony Weiner says his Twitter account w...,78,78_weiner_weiners_abedin_anthony,"[weiner, weiners, abedin, anthony, sexting, hi...",[Anthony Weiner will finally be cut off. The p...,weiner - weiners - abedin - anthony - sexting ...,0.530236,False
2151,The GOP staffer who called Malia and Sasha Oba...,-1,-1_brewing_to_the_in,"[brewing, to, the, in, and, of, that, was, is,...","[Update number 14 - July 8, 2015 Today, July 8...",brewing - to - the - in - and - of - that - wa...,0.000000,False


In [2]:
len(set(doc_info_df['Topic']))

262

In [4]:
len(all_data)

379

In [8]:
# -*- coding: utf-8 -*-
"""
File: run_multi_gpu_pipeline.py
Mô tả: Quy trình phân cụm văn bản, được tối ưu để chạy song song trên nhiều GPU.
"""

import json
import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch


def main():
    # =============================================================================
    # BƯỚC 1: CÁC THAM SỐ CẤU HÌNH
    # =============================================================================
    print("--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM SONG SONG TRÊN NHIỀU GPU ---")

    DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
    EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
    MAX_TOKENS_PER_DOC = 8192

    # Batch size này sẽ được áp dụng cho MỖI GPU.
    BATCH_SIZE = 8
    MIN_TOPIC_SIZE = 5
    OUTPUT_DIR = "results_multi_gpu_pipeline_bge"

    # Kiểm tra số lượng GPU có sẵn
    gpu_count = torch.cuda.device_count()
    print(f"Phát hiện thấy {gpu_count} GPU có sẵn.")
    DEVICE = "cuda" if gpu_count > 0 else "cpu"

    print(f"Thiết bị chính: {DEVICE.upper()} | Giới hạn token mỗi văn bản: {MAX_TOKENS_PER_DOC}")
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # =============================================================================
    # BƯỚC 2: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU
    # =============================================================================
    print(f"\n[BƯỚC 2] Đang tải và xử lý dữ liệu từ: {DATA_FILE_PATH}...")
    try:
        with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
    except Exception as e:
        print(f"[LỖI] Không thể đọc file dữ liệu. Lỗi: {e}")
        return

    docs = []
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    for item in all_data:
        if isinstance(item, str):
            sub_docs = item.split("|||||")
            for sub_doc in sub_docs:
                sub_doc = sub_doc.strip()
                if not sub_doc:
                    continue
                tokens = tokenizer.encode(sub_doc, add_special_tokens=False)
                if len(tokens) > MAX_TOKENS_PER_DOC:
                    tokens = tokens[:MAX_TOKENS_PER_DOC]
                processed_doc = tokenizer.decode(tokens)
                docs.append(processed_doc)
    print(f"[INFO] Tổng số văn bản sau khi xử lý: {len(docs)}")

    # =============================================================================
    # BƯỚC 3: TẢI MÔ HÌNH EMBEDDING
    # =============================================================================
    print(f"\n[BƯỚC 3] Đang tải mô hình embedding: {EMBEDDING_MODEL_NAME}...")
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cpu')
    print("[SUCCESS] Tải xong mô hình embedding.")

    # =============================================================================
    # BƯỚC 4: TẠO EMBEDDINGS SONG SONG TRÊN NHIỀU GPU
    # =============================================================================
    print(f"\n[BƯỚC 4] Bắt đầu tạo embeddings song song trên {gpu_count} GPU...")
    # Xác định danh sách thiết bị cho mỗi tiến trình
    target_devices = [f"cuda:{i}" for i in range(gpu_count)]
    # Tạo pool xử lý đa tiến trình
    pool = embedding_model.start_multi_process_pool(target_devices=target_devices)
    embeddings = embedding_model.encode_multi_process(
        docs,
        pool=pool,
        batch_size=BATCH_SIZE,
        chunk_size=BATCH_SIZE * 10
    )
    embedding_model.stop_multi_process_pool(pool)
    print(f"[SUCCESS] Đã tạo xong embeddings với shape: {embeddings.shape}")

    # =============================================================================
    # BƯỚC 5 & 6: HUẤN LUYỆN BERTOPIC VÀ LƯU KẾT QUẢ
    # =============================================================================
    print(f"\n[BƯỚC 5] Đang huấn luyện BERTopic...")
    topic_model = BERTopic(verbose=True, min_topic_size=MIN_TOPIC_SIZE)
    topics, probs = topic_model.fit_transform(docs, embeddings)

    print("\n[BƯỚC 6] Đang lưu kết quả...")
    doc_info_df = topic_model.get_document_info(docs)
    doc_info_df.to_csv(os.path.join(OUTPUT_DIR, "chi_tiet_van_ban_theo_chu_de.csv"), index=False, encoding='utf-8-sig')
    print(f"[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: {OUTPUT_DIR}")

    print("\n--- QUY TRÌNH HOÀN TẤT ---")


if __name__ == "__main__":
    main()


--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM SONG SONG TRÊN NHIỀU GPU ---
Phát hiện thấy 2 GPU có sẵn.
Thiết bị chính: CUDA | Giới hạn token mỗi văn bản: 8192

[BƯỚC 2] Đang tải và xử lý dữ liệu từ: /home/hungpv/projects/next_work/raw_data.json...


Token indices sequence length is longer than the specified maximum sequence length for this model (10349 > 8192). Running this sequence through the model will result in indexing errors


[INFO] Tổng số văn bản sau khi xử lý: 2153

[BƯỚC 3] Đang tải mô hình embedding: BAAI/bge-m3...
[SUCCESS] Tải xong mô hình embedding.

[BƯỚC 4] Bắt đầu tạo embeddings song song trên 2 GPU...


2025-06-23 10:58:42,007 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[SUCCESS] Đã tạo xong embeddings với shape: (2153, 1024)

[BƯỚC 5] Đang huấn luyện BERTopic...


2025-06-23 10:58:52,328 - BERTopic - Dimensionality - Completed ✓
2025-06-23 10:58:52,329 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-23 10:58:52,425 - BERTopic - Cluster - Completed ✓
2025-06-23 10:58:52,428 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-23 10:58:54,486 - BERTopic - Representation - Completed ✓



[BƯỚC 6] Đang lưu kết quả...
[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: results_multi_gpu_pipeline_bge

--- QUY TRÌNH HOÀN TẤT ---


In [2]:
import torch
gpu_count = torch.cuda.device_count()

In [3]:
gpu_count

2

In [ ]:
# -*- coding: utf-8 -*-
"""
File: run_multi_gpu_pipeline.py
Mô tả: Quy trình phân cụm văn bản, được tối ưu để chạy song song trên nhiều GPU và giải phóng bộ nhớ GPU sau mỗi phần.
"""

import json
import os
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch


def main():
    # =============================================================================
    # BƯỚC 1: CÁC THAM SỐ CẤU HÌNH
    # =============================================================================
    print("--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM SONG SONG TRÊN NHIỀU GPU ---")

    DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
    EMBEDDING_MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"
    MAX_TOKENS_PER_DOC = 10000

    # Batch size này sẽ được áp dụng cho MỖI GPU.
    BATCH_SIZE = 8
    MIN_TOPIC_SIZE = 5
    OUTPUT_DIR = "results_multi_gpu_pipeline"

    # Số văn bản xử lý mỗi lượt để tránh tràn bộ nhớ
    CHUNK_DOCS = 500

    # Kiểm tra số lượng GPU có sẵn
    gpu_count = torch.cuda.device_count()
    print(f"Phát hiện thấy {gpu_count} GPU có sẵn.")
    DEVICE = "cuda" if gpu_count > 0 else "cpu"

    print(f"Thiết bị chính: {DEVICE.upper()} | Giới hạn token mỗi văn bản: {MAX_TOKENS_PER_DOC}")
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # =============================================================================
    # BƯỚC 2: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU
    # =============================================================================
    print(f"\n[BƯỚC 2] Đang tải và xử lý dữ liệu từ: {DATA_FILE_PATH}...")
    try:
        with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
    except Exception as e:
        print(f"[LỖI] Không thể đọc file dữ liệu. Lỗi: {e}")
        return

    docs = []
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    for item in all_data:
        if isinstance(item, str):
            sub_docs = item.split("|||||")
            for sub_doc in sub_docs:
                sub_doc = sub_doc.strip()
                if not sub_doc:
                    continue
                tokens = tokenizer.encode(sub_doc, add_special_tokens=False)
                if len(tokens) > MAX_TOKENS_PER_DOC:
                    tokens = tokens[:MAX_TOKENS_PER_DOC]
                processed_doc = tokenizer.decode(tokens)
                docs.append(processed_doc)
    print(f"[INFO] Tổng số văn bản sau khi xử lý: {len(docs)}")

    # =============================================================================
    # BƯỚC 3: TẢI MÔ HÌNH EMBEDDING
    # =============================================================================
    print(f"\n[BƯỚC 3] Đang tải mô hình embedding: {EMBEDDING_MODEL_NAME}...")
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cpu')
    print("[SUCCESS] Tải xong mô hình embedding.")

    # =============================================================================
    # BƯỚC 4: TẠO EMBEDDINGS CHUNG VÀ GIẢI PHÓNG BỘ NHỚ GPU THEO CHÙM
    # =============================================================================
    print(f"\n[BƯỚC 4] Bắt đầu tạo embeddings qua {gpu_count} GPU với giải phóng bộ nhớ sau mỗi phần...")
    target_devices = [f"cuda:{i}" for i in range(gpu_count)]
    embeddings_parts = []

    for start in range(0, len(docs), CHUNK_DOCS):
        end = min(start + CHUNK_DOCS, len(docs))
        chunk = docs[start:end]
        print(f"[INFO] Xử lý văn bản từ index {start} đến {end}...")

        # Khởi pool cho mỗi phần để đảm bảo giải phóng GPU sau khi stop
        pool = embedding_model.start_multi_process_pool(target_devices=target_devices)
        sub_embeds = embedding_model.encode_multi_process(
            chunk,
            pool=pool,
            batch_size=BATCH_SIZE,
            chunk_size=BATCH_SIZE * 5
        )
        embedding_model.stop_multi_process_pool(pool)

        embeddings_parts.append(sub_embeds)

        # Giải phóng GPU cache
        torch.cuda.empty_cache()
        del sub_embeds, pool

    # Kết hợp lại embeddings toàn bộ
    embeddings = np.vstack(embeddings_parts)
    print(f"[SUCCESS] Đã tạo xong embeddings với shape: {embeddings.shape}")

    # =============================================================================
    # BƯỚC 5 & 6: HUẤN LUYỆN BERTOPIC VÀ LƯU KẾT QUẢ
    # =============================================================================
    print(f"\n[BƯỚC 5] Đang huấn luyện BERTopic...")
    topic_model = BERTopic(verbose=True, min_topic_size=MIN_TOPIC_SIZE)
    topics, probs = topic_model.fit_transform(docs, embeddings)

    print("\n[BƯỚC 6] Đang lưu kết quả...")
    doc_info_df = topic_model.get_document_info(docs)
    doc_info_df.to_csv(
        os.path.join(OUTPUT_DIR, "chi_tiet_van_ban_theo_chu_de.csv"),
        index=False,
        encoding='utf-8-sig'
    )
    print(f"[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: {OUTPUT_DIR}")

    print("\n--- QUY TRÌNH HOÀN TẤT ---")


if __name__ == "__main__":
    main()


In [2]:
df = pd.read_csv("/home/hungpv/projects/next_work/results_multi_gpu_pipeline_bge/chi_tiet_van_ban_theo_chu_de.csv", encoding='utf-8-sig')

In [5]:
df

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,CLOSE Sierra Nevada Brewing Co. is releasing a...,51,51_brewing_company_co_brewery,"['brewing', 'company', 'co', 'brewery', 'beer'...","['November 21, 2018 The Camp Fire–the most dev...",brewing - company - co - brewery - beer - town...,0.838920,True
1,Sierra Nevada Brewery in Chico is brewing up a...,51,51_brewing_company_co_brewery,"['brewing', 'company', 'co', 'brewery', 'beer'...","['November 21, 2018 The Camp Fire–the most dev...",brewing - company - co - brewery - beer - town...,0.816194,False
2,"November 21, 2018 The Camp Fire–the most devas...",51,51_brewing_company_co_brewery,"['brewing', 'company', 'co', 'brewery', 'beer'...","['November 21, 2018 The Camp Fire–the most dev...",brewing - company - co - brewery - beer - town...,0.799550,True
3,"In early November, the Camp Fire swept across ...",51,51_brewing_company_co_brewery,"['brewing', 'company', 'co', 'brewery', 'beer'...","['November 21, 2018 The Camp Fire–the most dev...",brewing - company - co - brewery - beer - town...,0.765014,False
4,"(Packaged in cans by Sierra Nevada, and draugh...",51,51_brewing_company_co_brewery,"['brewing', 'company', 'co', 'brewery', 'beer'...","['November 21, 2018 The Camp Fire–the most dev...",brewing - company - co - brewery - beer - town...,0.832759,True
...,...,...,...,...,...,...,...,...
2148,"Rep. Steve Cohen, a Tennessee Democrat, attrac...",59,59_weiner_weiners_abedin_anthony,"['weiner', 'weiners', 'abedin', 'anthony', 'se...","[""Anthony Weiner will finally be cut off. The ...",weiner - weiners - abedin - anthony - sexting ...,0.658563,False
2149,Porn Star Says Weiner Sent Her a Private Messa...,59,59_weiner_weiners_abedin_anthony,"['weiner', 'weiners', 'abedin', 'anthony', 'se...","[""Anthony Weiner will finally be cut off. The ...",weiner - weiners - abedin - anthony - sexting ...,0.235362,False
2150,Rep. Anthony Weiner says his Twitter account w...,59,59_weiner_weiners_abedin_anthony,"['weiner', 'weiners', 'abedin', 'anthony', 'se...","[""Anthony Weiner will finally be cut off. The ...",weiner - weiners - abedin - anthony - sexting ...,0.783293,False
2151,The GOP staffer who called Malia and Sasha Oba...,33,33_melania_forest_llamas_singular,"['melania', 'forest', 'llamas', 'singular', 't...",['You’ve probably come across the singular pro...,melania - forest - llamas - singular - tom - s...,1.000000,False


In [4]:
len(set(df['Topic']))

172

In [7]:
with open("/home/hungpv/projects/next_work/raw_data.json",'r') as f:
    all_data = json.load(f)
len(all_data)

379

# Kết quả tốt nhất hiện tại

In [4]:
# -*- coding: utf-8 -*-
"""
File: run_multi_gpu_advanced_tuning.py
Mô tả: Quy trình phân cụm văn bản hoàn chỉnh, kết hợp cả hai kỹ thuật nâng cao:
       1. Chạy song song trên nhiều GPU để tăng tốc độ tạo embedding.
       2. Tinh chỉnh sâu các tham số của UMAP và HDBSCAN để cải thiện chất lượng phân cụm.
"""

import json
import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch

# --- Thêm import cho UMAP và HDBSCAN ---
# from umap import UMAP
from umap.umap_ import UMAP

from hdbscan import HDBSCAN


def main():
    # =============================================================================
    # BƯỚC 1: CÁC THAM SỐ CẤU HÌNH (Trung tâm điều khiển của bạn)
    # =============================================================================
    print("--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM NÂNG CAO TRÊN NHIỀU GPU ---")

    # --- Cấu hình file và dữ liệu ---
    DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
    EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
    MAX_TOKENS_PER_DOC = 8192

    # --- Cấu hình chạy song song ---
    BATCH_SIZE = 8 # Batch size này sẽ được áp dụng cho MỖI GPU
    
    # --- Cấu hình tinh chỉnh NÂNG CAO ---
    # -- UMAP (Giảm chiều) --
    UMAP_N_NEIGHBORS = 15
    UMAP_N_COMPONENTS = 10
    
    # -- HDBSCAN (Phân cụm) --
    # `HDBSCAN_MIN_CLUSTER_SIZE` sẽ thay thế cho `MIN_TOPIC_SIZE`
    HDBSCAN_MIN_CLUSTER_SIZE = 5
    HDBSCAN_MIN_SAMPLES = 2 # Giảm để có ít topic -1 (outlier) hơn

    # --- Cấu hình lưu trữ ---
    OUTPUT_DIR = "results_multi_gpu_advanced"

    # In ra cấu hình để tiện theo dõi
    print("\n--- Cấu hình thử nghiệm ---")
    gpu_count = torch.cuda.device_count()
    print(f"  Số GPU sử dụng: {gpu_count}")
    print(f"  Mô hình Embedding: {EMBEDDING_MODEL_NAME}")
    print(f"  UMAP n_neighbors: {UMAP_N_NEIGHBORS} | n_components: {UMAP_N_COMPONENTS}")
    print(f"  HDBSCAN min_cluster_size: {HDBSCAN_MIN_CLUSTER_SIZE} | min_samples: {HDBSCAN_MIN_SAMPLES}")
    print("--------------------------")
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # =============================================================================
    # BƯỚC 2: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU (Logic Truncation)
    # =============================================================================
    print(f"\n[BƯỚC 2] Đang tải và xử lý dữ liệu từ: {DATA_FILE_PATH}...")
    try:
        with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f: all_data = json.load(f)
    except Exception as e: print(f"[LỖI] Đọc file thất bại: {e}"); return
    
    docs = []
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    for item in all_data:
        if isinstance(item, str):
            sub_docs = item.split("|||||")
            for sub_doc in sub_docs:
                if not sub_doc.strip(): continue
                tokens = tokenizer.encode(sub_doc.strip(), add_special_tokens=False)
                if len(tokens) > MAX_TOKENS_PER_DOC: tokens = tokens[:MAX_TOKENS_PER_DOC]
                docs.append(tokenizer.decode(tokens))
    print(f"[INFO] Tổng số văn bản sau khi xử lý: {len(docs)}")

    # =============================================================================
    # BƯỚC 3: TẢI MÔ HÌNH EMBEDDING
    # =============================================================================
    print(f"\n[BƯỚC 3] Đang tải mô hình embedding...")
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cpu')
    print("[SUCCESS] Tải xong mô hình embedding.")

    # =============================================================================
    # BƯỚC 4: TẠO EMBEDDINGS SONG SONG TRÊN NHIỀU GPU
    # =============================================================================
    print(f"\n[BƯỚC 4] Bắt đầu tạo embeddings song song trên {gpu_count} GPU...")
    pool = embedding_model.start_multi_process_pool()
    embeddings = embedding_model.encode_multi_process(docs, pool=pool, batch_size=BATCH_SIZE)
    embedding_model.stop_multi_process_pool(pool)
    print(f"[SUCCESS] Đã tạo xong embeddings với shape: {embeddings.shape}")

    # =============================================================================
    # BƯỚC 5: KHỞI TẠO CÁC THÀNH PHẦN TÙY CHỈNH
    # =============================================================================
    print("\n[BƯỚC 5] Đang cấu hình UMAP và HDBSCAN...")
    umap_model = UMAP(n_neighbors=UMAP_N_NEIGHBORS, n_components=UMAP_N_COMPONENTS, min_dist=0.0, metric='cosine', random_state=42)
    hdbscan_model = HDBSCAN(min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE, min_samples=HDBSCAN_MIN_SAMPLES, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

    # =============================================================================
    # BƯỚC 6: HUẤN LUYỆN BERTOPIC VỚI CÁC THÀNH PHẦN TÙY CHỈNH
    # =============================================================================
    print(f"\n[BƯỚC 6] Đang huấn luyện BERTopic...")
    topic_model = BERTopic(
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        verbose=True
    )
    topics, probs = topic_model.fit_transform(docs, embeddings)
    print("[SUCCESS] Hoàn thành huấn luyện.")
    
    # =============================================================================
    # BƯỚC 7: LƯU KẾT QUẢ
    # =============================================================================
    print("\n[BƯỚC 7] Đang lưu kết quả...")
    doc_info_df = topic_model.get_document_info(docs)
    doc_info_df.to_csv(os.path.join(OUTPUT_DIR, "chi_tiet_van_ban_theo_chu_de.csv"), index=False, encoding='utf-8-sig')
    print(f"[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: {OUTPUT_DIR}")

    print("\n--- QUY TRÌNH HOÀN TẤT ---")


if __name__ == "__main__":
    main()

--- BẮT ĐẦU QUY TRÌNH PHÂN CỤM NÂNG CAO TRÊN NHIỀU GPU ---

--- Cấu hình thử nghiệm ---
  Số GPU sử dụng: 2
  Mô hình Embedding: BAAI/bge-m3
  UMAP n_neighbors: 15 | n_components: 10
  HDBSCAN min_cluster_size: 5 | min_samples: 2
--------------------------

[BƯỚC 2] Đang tải và xử lý dữ liệu từ: /home/hungpv/projects/next_work/raw_data.json...


Token indices sequence length is longer than the specified maximum sequence length for this model (10349 > 8192). Running this sequence through the model will result in indexing errors


[INFO] Tổng số văn bản sau khi xử lý: 2153

[BƯỚC 3] Đang tải mô hình embedding...
[SUCCESS] Tải xong mô hình embedding.

[BƯỚC 4] Bắt đầu tạo embeddings song song trên 2 GPU...


2025-06-24 07:22:32,265 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[SUCCESS] Đã tạo xong embeddings với shape: (2153, 1024)

[BƯỚC 5] Đang cấu hình UMAP và HDBSCAN...

[BƯỚC 6] Đang huấn luyện BERTopic...


2025-06-24 07:22:47,250 - BERTopic - Dimensionality - Completed ✓
2025-06-24 07:22:47,251 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-24 07:22:47,390 - BERTopic - Cluster - Completed ✓
2025-06-24 07:22:47,393 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-24 07:22:49,540 - BERTopic - Representation - Completed ✓


[SUCCESS] Hoàn thành huấn luyện.

[BƯỚC 7] Đang lưu kết quả...
[SUCCESS] Đã lưu kết quả phân tích ra file CSV trong thư mục: results_multi_gpu_advanced

--- QUY TRÌNH HOÀN TẤT ---


In [5]:
df = pd.read_csv("/home/hungpv/projects/next_work/results_multi_gpu_advanced/chi_tiet_van_ban_theo_chu_de.csv", encoding='utf-8-sig')

In [6]:
len(set(df['Topic']))

271

In [11]:
# File: evaluate_from_files_v2.py
import json
import os
import pandas as pd
import numpy as np
from sklearn import metrics
from transformers import AutoTokenizer # <<< THÊM IMPORT NÀY

def run_benchmark_external(true_labels, predicted_labels):
    """Hàm này giữ nguyên, không thay đổi."""
    print("--- BẮT ĐẦU ĐÁNH GIÁ BENCHMARK PHÂN CỤM ---")
    # ... (code bên trong hàm này giữ nguyên như trước) ...
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)
    non_outlier_mask = predicted_labels != -1
    filtered_true_labels = true_labels[non_outlier_mask]
    filtered_predicted_labels = predicted_labels[non_outlier_mask]
    outlier_percentage = 100 * (1 - (np.sum(non_outlier_mask) / len(predicted_labels)))
    print(f"Tỷ lệ Outlier (nhãn -1): {outlier_percentage:.2f}%")
    print(f"Số điểm được dùng để đánh giá: {len(filtered_true_labels)} / {len(true_labels)}")
    if len(filtered_true_labels) == 0:
        print("Không có điểm nào được phân cụm (tất cả là outlier).")
        return
    ari_score = metrics.adjusted_rand_score(filtered_true_labels, filtered_predicted_labels)
    ami_score = metrics.adjusted_mutual_info_score(filtered_true_labels, filtered_predicted_labels)
    homogeneity, completeness, v_measure = metrics.homogeneity_completeness_v_measure(filtered_true_labels, filtered_predicted_labels)
    print("\n--- KẾT QUẢ ĐÁNH GIÁ NGOẠI KIỂM ---")
    print(f"Adjusted Rand Index (ARI):      {ari_score:.4f} (Càng gần 1 càng tốt)")
    print(f"Adjusted Mutual Information (AMI): {ami_score:.4f} (Càng gần 1 càng tốt)")
    print(f"Homogeneity (Tính đồng nhất):     {homogeneity:.4f}")
    print(f"Completeness (Tính đầy đủ):      {completeness:.4f}")
    print(f"V-measure (Trung bình H & C):    {v_measure:.4f}")
    print("---------------------------------------")


def align_and_evaluate(ground_truth_path, bertopic_csv_path, model_name, max_tokens):
    """
    Hàm chính đã được cập nhật để xử lý logic truncation.
    """
    # --- Đọc dữ liệu ---
    print(f"Đọc nhãn gốc từ: {ground_truth_path}")
    with open(ground_truth_path, 'r', encoding='utf-8') as f:
        true_clusters_list = json.load(f)

    print(f"Đọc kết quả dự đoán từ: {bertopic_csv_path}")
    predictions_df = pd.read_csv(bertopic_csv_path)

    # --- Tải tokenizer để thực hiện truncation ---
    print(f"Tải tokenizer cho model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # --- Bước 1: Tạo bảng ánh xạ từ VĂN BẢN ĐÃ CẮT NGẮN -> nhãn gốc ---
    print("Tạo bảng ánh xạ từ nhãn gốc (có áp dụng truncation)...")
    doc_to_true_label = {}
    for i, cluster_docs in enumerate(true_clusters_list):
        true_label = f"True_Cluster_{i}"
        for full_doc in cluster_docs:
            full_doc_stripped = full_doc.strip()
            if not full_doc_stripped: continue

            # === LOGIC MỚI: ÁP DỤNG CÙNG CÁCH TRUNCATION CHO DỮ LIỆU GỐC ===
            tokens = tokenizer.encode(full_doc_stripped, add_special_tokens=False)
            if len(tokens) > max_tokens:
                tokens = tokens[:max_tokens]
            
            truncated_doc = tokenizer.decode(tokens)
            # Dùng văn bản đã được cắt ngắn làm key
            doc_to_true_label[truncated_doc.strip()] = true_label
            # =============================================================

    # --- Bước 2: Tạo danh sách nhãn gốc và nhãn dự đoán tương ứng ---
    # Lấy danh sách văn bản (đã bị cắt ngắn) từ file kết quả của BERTopic
    all_docs_from_pred = predictions_df['Document'].tolist()
    
    true_labels = []
    unmatched_count = 0
    # Ánh xạ từ văn bản trong kết quả của BERTopic sang nhãn gốc
    for doc in all_docs_from_pred:
        # doc.strip() ở đây sẽ khớp với key đã được truncate trong `doc_to_true_label`
        true_label = doc_to_true_label.get(doc.strip(), "N/A_Unlabeled")
        if true_label == "N/A_Unlabeled":
            unmatched_count += 1
        true_labels.append(true_label)
        
    print(f"Số văn bản không tìm thấy trong nhãn gốc: {unmatched_count}")
        
    # Lấy danh sách nhãn dự đoán trực tiếp từ DataFrame
    predicted_labels = predictions_df['Topic'].tolist()

    if len(true_labels) != len(predicted_labels):
        print("[LỖI] Số lượng nhãn không khớp!")
        return

    # --- Bước 3: Chạy Benchmark ---
    run_benchmark_external(true_labels, predicted_labels)


if __name__ == '__main__':
    # --- CẤU HÌNH ---
    # !!! QUAN TRỌNG: Các tham số này PHẢI KHỚP với các tham số bạn đã dùng để huấn luyện
    EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
    MAX_TOKENS_PER_DOC = 8192
    
    # --- Đường dẫn file ---
    GROUND_TRUTH_PATH = '/home/hungpv/projects/next_work/grouth_truth_cluster.json'
    BERTOPIC_CSV_PATH = '/home/hungpv/projects/next_work/results_multi_gpu_advanced/chi_tiet_van_ban_theo_chu_de.csv'
    
    if not os.path.exists(GROUND_TRUTH_PATH) or not os.path.exists(BERTOPIC_CSV_PATH):
        print("[LỖI] Không tìm thấy file đầu vào. Vui lòng kiểm tra lại đường dẫn.")
    else:
        align_and_evaluate(GROUND_TRUTH_PATH, BERTOPIC_CSV_PATH, EMBEDDING_MODEL_NAME, MAX_TOKENS_PER_DOC)

Đọc nhãn gốc từ: /home/hungpv/projects/next_work/grouth_truth_cluster.json
Đọc kết quả dự đoán từ: /home/hungpv/projects/next_work/results_multi_gpu_advanced/chi_tiet_van_ban_theo_chu_de.csv
Tải tokenizer cho model: BAAI/bge-m3


Token indices sequence length is longer than the specified maximum sequence length for this model (10349 > 8192). Running this sequence through the model will result in indexing errors


Tạo bảng ánh xạ từ nhãn gốc (có áp dụng truncation)...
Số văn bản không tìm thấy trong nhãn gốc: 0
--- BẮT ĐẦU ĐÁNH GIÁ BENCHMARK PHÂN CỤM ---
Tỷ lệ Outlier (nhãn -1): 5.34%
Số điểm được dùng để đánh giá: 2038 / 2153

--- KẾT QUẢ ĐÁNH GIÁ NGOẠI KIỂM ---
Adjusted Rand Index (ARI):      0.6805 (Càng gần 1 càng tốt)
Adjusted Mutual Information (AMI): 0.8455 (Càng gần 1 càng tốt)
Homogeneity (Tính đồng nhất):     0.9176
Completeness (Tính đầy đủ):      0.9812
V-measure (Trung bình H & C):    0.9483
---------------------------------------


In [9]:

with open('/home/hungpv/projects/next_work/raw_data.json', 'r', encoding='utf-8') as f: all_data = json.load(f)


docs = []

for item in all_data:
    if isinstance(item, str):
        sub_docs = item.split("|||||")
        sub = []
        for sub_doc in sub_docs:
            sub_doc = sub_doc.strip()
            sub.append(sub_doc)
        docs.append(sub)
        
with open("/home/hungpv/projects/next_work/grouth_truth_cluster.json",'w', encoding='utf-8') as f:
    json.dump(docs,f,indent=4,ensure_ascii=False)

# Grid Search Param

In [15]:
# -*- coding: utf-8 -*-
"""
File: grid_search_bertopic.py
Mô tả: Tự động hóa việc tìm kiếm tham số tốt nhất cho BERTopic bằng Grid Search.
       Script sẽ chạy nhiều tổ hợp tham số UMAP/HDBSCAN, đánh giá từng cái,
       và báo cáo kết quả tổng hợp cùng với bộ tham số tốt nhất.
"""

import json
import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch
# from umap import UMAP
from umap.umap_ import UMAP

from hdbscan import HDBSCAN
from sklearn import metrics
from sklearn.model_selection import ParameterGrid # <<< Thêm import này

def load_and_preprocess_data(data_path, model_name, max_tokens):
    """Tải và xử lý dữ liệu từ file."""
    print(f"\n[INFO] Đang tải và xử lý dữ liệu từ: {data_path}...")
    try:
        with open(data_path, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
    except Exception as e:
        print(f"[LỖI] Đọc file thất bại: {e}")
        return None
    
    docs = []
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    for item in all_data:
        if isinstance(item, str):
            sub_docs = item.split("|||||")
            for sub_doc in sub_docs:
                if not sub_doc.strip(): continue
                tokens = tokenizer.encode(sub_doc.strip(), add_special_tokens=False)
                if len(tokens) > max_tokens: tokens = tokens[:max_tokens]
                docs.append(tokenizer.decode(tokens))
    print(f"[INFO] Tổng số văn bản sau khi xử lý: {len(docs)}")
    return docs

def create_embeddings(docs, model_name, batch_size):
    """Tạo embedding song song trên nhiều GPU."""
    print(f"\n[INFO] Đang tải mô hình embedding: {model_name}...")
    embedding_model = SentenceTransformer(model_name, device='cpu')
    
    gpu_count = torch.cuda.device_count()
    print(f"\n[INFO] Bắt đầu tạo embeddings song song trên {gpu_count} GPU...")
    pool = embedding_model.start_multi_process_pool()
    embeddings = embedding_model.encode_multi_process(docs, pool=pool, batch_size=batch_size)
    embedding_model.stop_multi_process_pool(pool)
    print(f"[SUCCESS] Đã tạo xong embeddings với shape: {embeddings.shape}")
    return embeddings

def evaluate_run(true_labels, predicted_labels):
    """Hàm đánh giá và TRẢ VỀ một dictionary chứa các điểm số."""
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)
    non_outlier_mask = predicted_labels != -1
    
    # Nếu tất cả là outlier, trả về điểm số tệ nhất
    if np.sum(non_outlier_mask) == 0:
        return {"ARI": -1, "AMI": -1, "Homogeneity": -1, "Completeness": -1, "V-measure": -1, "Outlier %": 100.0}

    filtered_true = true_labels[non_outlier_mask]
    filtered_pred = predicted_labels[non_outlier_mask]
    
    scores = {
        "ARI": metrics.adjusted_rand_score(filtered_true, filtered_pred),
        "AMI": metrics.adjusted_mutual_info_score(filtered_true, filtered_pred),
        "Homogeneity": metrics.homogeneity_score(filtered_true, filtered_pred),
        "Completeness": metrics.completeness_score(filtered_true, filtered_pred),
        "V-measure": metrics.v_measure_score(filtered_true, filtered_pred),
        "Outlier %": 100 * (1 - (np.sum(non_outlier_mask) / len(predicted_labels)))
    }
    return scores

def main():
    # =============================================================================
    # BƯỚC 1: CÁC THAM SỐ CẤU HÌNH VÀ LƯỚI TÌM KIẾM
    # =============================================================================
    # --- Cấu hình tĩnh ---
    DATA_FILE_PATH = "/home/hungpv/projects/next_work/raw_data.json"
    GROUND_TRUTH_PATH = "/home/hungpv/projects/next_work/grouth_truth_cluster.json"
    EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
    MAX_TOKENS_PER_DOC = 8192
    BATCH_SIZE = 16
    OUTPUT_DIR = "results_grid_search"
    PRIMARY_METRIC = "AMI" # Chỉ số dùng để chọn model tốt nhất

    # --- ĐỊNH NGHĨA LƯỚI THAM SỐ ĐỂ TÌM KIẾM ---
# --- ĐỊNH NGHĨA LƯỚI THAM SỐ ĐÃ ĐƯỢC TINH CHỈNH ---
    param_grid = {
        # Ưu tiên các giá trị thấp để tập trung vào cấu trúc local
        'UMAP_N_NEIGHBORS': [5, 10, 15], 
        
        # Giữ nguyên để khám phá
        'UMAP_N_COMPONENTS': [5, 10], 
        
        # Neo quanh con số bạn biết (5-10 docs)
        'HDBSCAN_MIN_CLUSTER_SIZE': [5, 8], 
        
        # Chỉ dùng các giá trị rất thấp để tối đa độ nhạy
        'HDBSCAN_MIN_SAMPLES': [1, 2, 3] 
    }
    
    print("--- BẮT ĐẦU QUY TRÌNH GRID SEARCH CHO BERTOPIC ---")
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # =============================================================================
    # BƯỚC 2: CHUẨN BỊ DỮ LIỆU (CHẠY 1 LẦN)
    # =============================================================================
    docs = load_and_preprocess_data(DATA_FILE_PATH, EMBEDDING_MODEL_NAME, MAX_TOKENS_PER_DOC)
    embeddings = create_embeddings(docs, EMBEDDING_MODEL_NAME, BATCH_SIZE)
    
    # Tải và xử lý ground truth để đánh giá
    with open(GROUND_TRUTH_PATH, 'r', encoding='utf-8') as f: true_clusters_list = json.load(f)
    doc_to_true_label = {}
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    for i, cluster_docs in enumerate(true_clusters_list):
        for doc in cluster_docs:
            tokens = tokenizer.encode(doc.strip(), add_special_tokens=False)
            if len(tokens) > MAX_TOKENS_PER_DOC: tokens = tokens[:MAX_TOKENS_PER_DOC]
            doc_to_true_label[tokenizer.decode(tokens).strip()] = f"True_Cluster_{i}"
    true_labels = [doc_to_true_label.get(doc.strip(), "N/A_Unlabeled") for doc in docs]

    # =============================================================================
    # BƯỚC 3: VÒNG LẶP GRID SEARCH
    # =============================================================================
    grid = ParameterGrid(param_grid)
    results = []
    best_score = -1
    best_params = None

    print(f"\n[INFO] Bắt đầu tìm kiếm trên {len(grid)} tổ hợp tham số...")

    for i, params in enumerate(grid):
        print(f"\n--- Thử nghiệm {i+1}/{len(grid)}: {params} ---")
        
        try:
            # Khởi tạo các thành phần với tham số hiện tại
            umap_model = UMAP(n_neighbors=params['UMAP_N_NEIGHBORS'], n_components=params['UMAP_N_COMPONENTS'], min_dist=0.0, metric='cosine', random_state=42)
            hdbscan_model = HDBSCAN(min_cluster_size=params['HDBSCAN_MIN_CLUSTER_SIZE'], min_samples=params['HDBSCAN_MIN_SAMPLES'], metric='euclidean', cluster_selection_method='eom', prediction_data=True)
            
            # Khởi tạo và huấn luyện BERTopic
            topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, verbose=False)
            topics, _ = topic_model.fit_transform(docs, embeddings)

            # Đánh giá kết quả
            scores = evaluate_run(true_labels, topics)
            
            # Lưu kết quả
            current_run = {**params, **scores}
            results.append(current_run)
            
            # Cập nhật kết quả tốt nhất
            if scores[PRIMARY_METRIC] > best_score:
                best_score = scores[PRIMARY_METRIC]
                best_params = params
                print(f"🔥🔥🔥 New best score found! {PRIMARY_METRIC}: {best_score:.4f}")

        except Exception as e:
            print(f"[LỖI] Gặp lỗi với tham số {params}. Lỗi: {e}")
            results.append({**params, "ARI": "Error", "AMI": "Error"})

    # =============================================================================
    # BƯỚC 4: BÁO CÁO KẾT QUẢ
    # =============================================================================
    print("\n\n--- HOÀN TẤT GRID SEARCH ---")
    
    # Chuyển kết quả sang DataFrame để dễ xem
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by=PRIMARY_METRIC, ascending=False)
    
    # Lưu bảng tổng hợp kết quả
    results_df.to_csv(os.path.join(OUTPUT_DIR, "grid_search_summary.csv"), index=False, encoding='utf-8-sig')
    
    print(f"\nBảng tổng hợp kết quả đã được lưu tại: {os.path.join(OUTPUT_DIR, 'grid_search_summary.csv')}")
    
    print("\n--- Top 5 cấu hình tốt nhất ---")
    print(results_df.head(5))
    
    print("\n--- Cấu hình tốt nhất được tìm thấy ---")
    print(f"Điểm {PRIMARY_METRIC} cao nhất: {best_score:.4f}")
    print("Với các tham số:")
    print(best_params)

if __name__ == "__main__":
    main()

--- BẮT ĐẦU QUY TRÌNH GRID SEARCH CHO BERTOPIC ---

[INFO] Đang tải và xử lý dữ liệu từ: /home/hungpv/projects/next_work/raw_data.json...


Token indices sequence length is longer than the specified maximum sequence length for this model (10349 > 8192). Running this sequence through the model will result in indexing errors


[INFO] Tổng số văn bản sau khi xử lý: 2153

[INFO] Đang tải mô hình embedding: BAAI/bge-m3...

[INFO] Bắt đầu tạo embeddings song song trên 2 GPU...
[SUCCESS] Đã tạo xong embeddings với shape: (2153, 1024)


Token indices sequence length is longer than the specified maximum sequence length for this model (10349 > 8192). Running this sequence through the model will result in indexing errors



[INFO] Bắt đầu tìm kiếm trên 36 tổ hợp tham số...

--- Thử nghiệm 1/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 1, 'UMAP_N_COMPONENTS': 5, 'UMAP_N_NEIGHBORS': 5} ---
🔥🔥🔥 New best score found! AMI: 0.8781

--- Thử nghiệm 2/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 1, 'UMAP_N_COMPONENTS': 5, 'UMAP_N_NEIGHBORS': 10} ---

--- Thử nghiệm 3/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 1, 'UMAP_N_COMPONENTS': 5, 'UMAP_N_NEIGHBORS': 15} ---

--- Thử nghiệm 4/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 1, 'UMAP_N_COMPONENTS': 10, 'UMAP_N_NEIGHBORS': 5} ---

--- Thử nghiệm 5/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 1, 'UMAP_N_COMPONENTS': 10, 'UMAP_N_NEIGHBORS': 10} ---

--- Thử nghiệm 6/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 1, 'UMAP_N_COMPONENTS': 10, 'UMAP_N_NEIGHBORS': 15} ---

--- Thử nghiệm 7/36: {'HDBSCAN_MIN_CLUSTER_SIZE': 5, 'HDBSCAN_MIN_SAMPLES': 2, 'UMAP_N_COMPONENTS': 5, 'UMAP_N_NEIGHB